In [1]:
import os, sys 
import pandas as pd
import numpy as np
os.chdir('../src')
import config
import glob
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.metrics import accuracy_score
import joblib
from utils import utils
from tqdm import tqdm

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [24]:
nn_folder = 'stacker_xgb_8'


gt_train = pd.read_csv(config.DATA_FOLDER + 'gt_train.csv')

sub_train = pd.read_csv(config.OUTPUT_FOLDER + '%s/training_dataset.csv'%nn_folder)
sub_train = sub_train.drop('target', 1 ).merge(gt_train[['id', 'target']], on = 'id')
assert len(gt_train) == len(sub_train)

In [21]:
splits = {}
for i in np.arange(5):
    splits[i] = {}
    for c in ['train', 'val']:
        splits[i][c] = utils.read_list_from_file(config.OUTPUT_FOLDER + 'nn_8' + '/splits/%s_%d.txt'%(c, i))

In [13]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_jobs=4)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=4, n_neighbors=5, p=2,
           weights='uniform')

In [31]:
sub_train['pred'] = np.nan

In [43]:
kn.fit(sub_train[sub_train['id'].isin(splits[4]['train'])].drop(['id', 'target', 'pred'], 1).values, 
       sub_train[sub_train['id'].isin(splits[4]['train'])]['target'].values)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=4, n_neighbors=5, p=2,
           weights='uniform')

In [44]:
sub_train.loc[sub_train['id'].isin(splits[4]['val']), 'pred'] = \
        kn.predict(sub_train[sub_train['id'].isin(splits[4]['val'])].drop(['id', 'target', 'pred'], 1).values)

In [35]:
sub_train.head()

,id,bed_nn_0,bird_nn_0,cat_nn_0,dog_nn_0,down_nn_0,eight_nn_0,five_nn_0,four_nn_0,go_nn_0,...,stop_nn_8,three_nn_8,tree_nn_8,two_nn_8,up_nn_8,wow_nn_8,yes_nn_8,zero_nn_8,target,pred
0,73af0c50_nohash_1_wow,1.352355e-06,6.142738e-07,1.147357e-07,0.000017,0.000267,1.399819e-08,0.000220,4.493637e-07,2.473998e-06,...,6.349922e-13,1.128661e-15,2.002880e-17,1.062139e-18,1.435348e-16,1.000000,2.870726e-16,4.077335e-12,28,28.0
1,187af8be_nohash_1_wow,8.155957e-05,3.822336e-06,2.571743e-06,0.000027,0.004799,2.005677e-07,0.000063,7.198362e-07,9.564436e-06,...,4.266974e-14,2.129133e-14,2.094194e-16,3.176394e-16,1.036997e-14,1.000000,3.887505e-15,6.675937e-12,28,28.0
2,c4533c47_nohash_1_wow,6.834891e-04,6.708858e-04,9.198749e-04,0.000299,0.000051,2.827419e-07,0.009776,5.383029e-05,8.697864e-07,...,4.788514e-07,1.481267e-12,6.625461e-14,8.544924e-14,6.501644e-09,0.999996,2.493418e-12,1.314932e-11,28,28.0
3,e57abea3_nohash_1_wow,1.370039e-06,1.721607e-06,9.491356e-07,0.000052,0.000460,5.194552e-08,0.000024,9.964987e-06,7.036098e-06,...,2.822602e-08,2.268823e-11,1.366700e-12,2.792973e-04,2.988625e-08,0.823557,3.043415e-09,1.209685e-07,28,28.0
4,baeac2ba_nohash_0_wow,4.550351e-07,1.435784e-07,1.666417e-07,0.000002,0.000057,2.337724e-09,0.000010,3.290563e-07,9.300914e-08,...,7.716369e-14,1.012390e-18,1.372269e-20,7.831374e-23,2.533955e-19,1.000000,5.542709e-16,2.603689e-16,28,28.0


In [25]:
df_cols = sub_train.drop(['id'], 1).columns.tolist()
for c in tqdm(config.mapping_dict.keys()):
    rc = [x for x in df_cols if c in x]
    sub_train[c] = sub_train[rc].mean(axis = 1)
    
sub_train['label'] = sub_train[config.mapping_dict.keys()].idxmax(axis = 1)

100%|██████████| 31/31 [00:09<00:00,  2.92it/s]


In [23]:
accuracy_score(sub_train['target'], sub_train['median_pred'])

0.96706064716140283

In [26]:
sub_train['fname'] = sub_train['id'] + '.wav'

In [27]:
sub_train[['fname', 'label']].to_csv('stacker_8_mean.csv', index = False)

In [13]:

def geo_mean(iterable):
    a = np.array(iterable)
    return a.prod()**(1.0/len(a))

In [45]:
accuracy_score(sub_train[sub_train['id'].isin(splits[4]['val'])]['target'],
               sub_train[sub_train['id'].isin(splits[4]['val'])]['pred'].astype(int))

0.95135606776703263